In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from sklearn.externals import joblib
import sklearn_recommender as skr
import sklearn_evaluation as Evaluation

C:\Users\sneha\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sneha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sneha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sneha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading packa

## LOAD DATA

In [2]:
# pip install sklearn-recommender
# pip install recommendation-system
# pip install sklearn-evaluation
# pip install evaluate
#Read userid,songid,listen_count triplets
triplets_file="https://static.turi.com/datasets/millionsong/10000.txt"
songs_metadata_file="https://static.turi.com/datasets/millionsong/song_data.csv"

In [3]:
song_1=pd.read_table(triplets_file,header=None)
song_1.columns={'user_id','song_id','listen_count'}

In [4]:
song_2=pd.read_csv(songs_metadata_file)

In [5]:
song_df=pd.merge(song_1,song_2.drop_duplicates(['song_id']),on="song_id",how="left")

## EXPLORATORY DATA ANALYSIS

In [6]:
song_df.head() 

,song_id,user_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,NaN,NaN,NaN,NaN
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,NaN,NaN,NaN,NaN
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,NaN,NaN,NaN,NaN
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,NaN,NaN,NaN,NaN
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,NaN,NaN,NaN,NaN


In [7]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000000 entries, 0 to 1999999
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   song_id       object 
 1   user_id       object 
 2   listen_count  int64  
 3   title         object 
 4   release       object 
 5   artist_name   object 
 6   year          float64
dtypes: float64(1), int64(1), object(5)
memory usage: 122.1+ MB


In [8]:
song_df.shape

(2000000, 7)

In [9]:
song_df.isna().sum()

song_id               0
user_id               0
listen_count          0
title           2000000
release         2000000
artist_name     2000000
year            2000000
dtype: int64

In [10]:
song_df=song_df.head(10000)

In [11]:
song_df.sample(3)

,song_id,user_id,listen_count,title,release,artist_name,year
1906,b6b799f34a204bd928ea014c243ddad6d0be4f8f,SOHEMBB12A6701E907,3,NaN,NaN,NaN,NaN
5072,eda12b54342243175dba3db83f66eade127d2b4e,SOLZCIF12A67AE0DF1,1,NaN,NaN,NaN,NaN
8835,e507a8acd03bbd1f6c2088bd70a0ab2ffb277cd7,SOGBXDU12AB018791F,1,NaN,NaN,NaN,NaN


In [12]:
# Concatenate song title and artist_name
song_df['song']=song_df['title'].map(str)+"-"+song_df['artist_name']

In [13]:
song_df.sample(10)

,song_id,user_id,listen_count,title,release,artist_name,year,song
8216,781d38e5e6029826707b207d7c2ef5c120fa36ed,SOTSHYT12A8C142868,1,NaN,NaN,NaN,NaN,NaN
6036,999902c7302ca251b23f5c0d643debce084076d3,SOIGAWN12A8151E078,1,NaN,NaN,NaN,NaN,NaN
3921,884209a41deb55df792f074bccf8af1c1c31768b,SOXMCEI12A8C132688,1,NaN,NaN,NaN,NaN,NaN
9608,c5c7548cba9970b695f1ade5164c11a0101eb00e,SOLWZVR12AB01849C6,3,NaN,NaN,NaN,NaN,NaN
3682,c2d86e9cbf756ce05ea3c163e24de394585b7c53,SOOXRJG12A8C13773E,1,NaN,NaN,NaN,NaN,NaN
5510,8caf9a87e266a22298bd977a63489d008af241c5,SOFRCGW12A81C21EA6,3,NaN,NaN,NaN,NaN,NaN
1753,ca80fbb6d0deb3cae53763099e2cae7306f005ec,SOWXVWT12A6D4F782E,1,NaN,NaN,NaN,NaN,NaN
5405,12a4a991c8b53cd6906995caed8b1f2bd3b6436a,SOFWMNN12A8C13C3F2,1,NaN,NaN,NaN,NaN,NaN
7434,4273611413c7199a53ee116edc8d7b0ba56482d5,SOTLEJS12A8C13C1E5,1,NaN,NaN,NaN,NaN,NaN
9971,bf03bbb01a7803ed1a5fec51bfe9423a79737d1a,SOJTJAU12A8C1395C4,15,NaN,NaN,NaN,NaN,NaN


In [14]:
song_df['user_id'].unique().sum()

'...'

In [15]:
len(song_df['song'].unique())

1

## MOST POPULAR SONGS

In [16]:
popular_song=song_df.groupby(['song']).agg({'listen_count':'count'}).reset_index()
listen_sum=popular_song['listen_count'].sum()

In [17]:
popular_song['Percentage']=popular_song['listen_count'].div(listen_sum)*100

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#To display all columns and rows

In [19]:
popular_song.sort_values(by=['listen_count'],ascending=False)

,song,listen_count,Percentage


## Train Test Split

In [20]:
train_data,test_data=train_test_split(song_df,test_size=0.30,random_state=0)

In [21]:
train_data.head()

,song_id,user_id,listen_count,title,release,artist_name,year,song
7681,f10e613636ee8e1f4d3a7f2b21ca9cd36d2e9d8d,SOOTBPW12A6701FBD5,2,NaN,NaN,NaN,NaN,NaN
9031,3fd3acaa8dfeb94b0602a33085b44ebe80545dd2,SOJHUDN12AF72A7ACC,1,NaN,NaN,NaN,NaN,NaN
3691,c2d86e9cbf756ce05ea3c163e24de394585b7c53,SOWHOTP12AB01885CD,1,NaN,NaN,NaN,NaN,NaN
202,17aa9f6dbdf753831da8f38c71b66b64373de613,SOBIMTY12A6D4F931F,1,NaN,NaN,NaN,NaN,NaN
5625,1b704d4cddabea8258bd93497fcb73eab32fa592,SOQFZDY12A8C1402DF,1,NaN,NaN,NaN,NaN,NaN


In [22]:
test_data.head()

,song_id,user_id,listen_count,title,release,artist_name,year,song
9394,97e48f0f188e04dcdb0f8e20a29dacb881d80c9e,SOLCAXX12B0B8079A5,1,NaN,NaN,NaN,NaN,NaN
898,a820d2d4f16bbd53be9e41e0417dfb234bfdfba8,SOHMTUC12A58A799D4,2,NaN,NaN,NaN,NaN,NaN
2398,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,SOEUTXF12A6D4FC6F8,2,NaN,NaN,NaN,NaN,NaN
5906,806ccae96c8ecb1c198482aff785ccd6bbe17143,SOIBSSJ12A8151E066,1,NaN,NaN,NaN,NaN,NaN
2343,8cbb5066924ec788e3fea9a4aae59586f46f38fa,SOPPROJ12AB0184E18,1,NaN,NaN,NaN,NaN,NaN


### Popularity based recommender class 

##### Now, based on the popularity of each song, create a recommender based on the training data.

In [32]:
import Recommenders 
import Evaluation
poprec=Recommenders.popularity_recommender_py()

In [33]:
poprec.create(train_data,'user_id','song')

In [40]:
user_id=users[10]
poprec.recommend(user_id)

NameError: name 'users' is not defined